# Lesson 6

In [2]:
from theano.sandbox import cuda
cuda.use('gpu0')

In [3]:
%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

Using Theano backend.


In [4]:
path = get_file('sherlock.txt', origin="https://sherlock-holm.es/stories/plain-text/cano.txt")
text = open(path).read()

print('corpus length:', len(text))

corpus length: 3868223


In [5]:
chars = sorted(list(set(text)))
vocab_size = len(chars)+1
print('total chars:', vocab_size)

total chars: 102


In [6]:
chars.insert(0, "\0")

In [7]:
''.join(chars[1:-6])

'\n !"&\'()*,-./0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]`abcdefghijklmnopqrstuvwxyz\x80\x99\x9f\xa0\xa2\xa3\xa8\xa9\xaa\xae\xb0\xb1\xb4\xb6'

In [8]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [9]:
idx = [char_indices[c] for c in text]

In [10]:
print (idx[:200])

[1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 46, 34, 31, 2, 29, 41, 39, 42, 38, 31, 46, 31, 2, 45, 34, 31, 44, 38, 41, 29, 37, 2, 34, 41, 38, 39, 31, 45, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 27, 73, 75, 63, 76, 73, 2, 29, 70, 69, 56, 69, 2, 30, 70, 80, 67, 60, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 46, 56, 57, 67, 60, 2, 70, 61, 2, 58, 70, 69, 75, 60, 69, 75, 74, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 27, 2, 45, 75, 76, 59, 80, 2, 35, 69, 2, 45, 58, 56, 73, 67, 60, 75, 1, 1, 2]


In [11]:
text[:200]

'\n\n\n\n                          THE COMPLETE SHERLOCK HOLMES\n\n                               Arthur Conan Doyle\n\n\n\n                                Table of contents\n\n               A Study In Scarlet\n\n '

In [12]:
print (text[:200])





                          THE COMPLETE SHERLOCK HOLMES

                               Arthur Conan Doyle



                                Table of contents

               A Study In Scarlet

 


# 3 Char Model

In [13]:
skip = 3

c1_data = [idx[i+0] for i in range(0, len(idx)-1-skip, skip)]
c2_data = [idx[i+1] for i in range(0, len(idx)-1-skip, skip)]
c3_data = [idx[i+2] for i in range(0, len(idx)-1-skip, skip)]
c4_data = [idx[i+3] for i in range(0, len(idx)-1-skip, skip)]

In [14]:
x1 = np.stack(c1_data)

In [15]:
x1.shape

(1289407,)

<br>Our inputs

In [16]:
x1 = np.stack(c1_data)
x2 = np.stack(c2_data)
x3 = np.stack(c3_data)

And output

In [17]:
y = np.stack(c4_data)

Let's check them out

In [18]:
x1[:4], x2[:4], x3[:4]

(array([1, 1, 2, 2]), array([1, 2, 2, 2]), array([1, 2, 2, 2]))

In [19]:
y[:4]

array([1, 2, 2, 2])

In [20]:
x1.shape, x2.shape, x3.shape, y.shape

((1289407,), (1289407,), (1289407,), (1289407,))

In [21]:
n_fac = 42

In [28]:
def embedding_input(name, n_in, n_out):
    inp = Input(shape=(1,), dtype='int64', name=name)
    emb = Embedding(n_in, n_out, input_length=1)(inp)
    return inp, Flatten()(emb)

In [29]:
c1_in, c1 = embedding_input('c1', vocab_size, n_fac)
c2_in, c2 = embedding_input('c2', vocab_size, n_fac)
c3_in, c3 = embedding_input('c3', vocab_size, n_fac)

### Model

In [30]:
n_hidden = 256

In [31]:
dense_in = Dense(n_hidden, activation='relu')

In [33]:
dense_hidden = Dense(n_hidden, activation='tanh')

In [37]:
dense_out = Dense(vocab_size, activation='softmax')

In [39]:
c1_hidden = dense_in(c1)
hidden_2 = dense_hidden(c1_hidden)

c2_dense = dense_in(c2)
c2_hidden = merge([c2_dense, hidden_2])
hidden_3 = dense_hidden(c2_hidden)

c3_dense = dense_in(c3)
c3_hidden = merge([c3_dense, hidden_2])

c4_out = dense_out(c3_hidden)

In [40]:
model = Model([c1_in, c2_in, c3_in], c4_out)

In [41]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [42]:
model.optimizer.lr=0.000001

In [43]:
model.fit([x1, x2, x3], y, batch_size=64, nb_epoch=4)

Epoch 1/4
1289407/1289407 [==============================] - 76s - loss: 4.1505    
Epoch 2/4
1289407/1289407 [==============================] - 76s - loss: 3.0836    
Epoch 3/4
1289407/1289407 [==============================] - 75s - loss: 2.9401    
Epoch 4/4
1289407/1289407 [==============================] - 76s - loss: 2.8935    


In [57]:
def get_next(inp):
    idxs = [char_indices[c] for c in inp]

#     arrs = [np.array(i)[np.newaxis] for i in idxs]
    arrs = [np.stack([i]) for i in idxs]

#     return arrs
    p = model.predict(arrs)
    i = np.argmax(p)
    return chars[i]

In [62]:
get_next('She')

' '

In [63]:
get_next('Hol')

' '

In [64]:
get_next('Wat')

' '

In [65]:
model.optimizer.lr=0.01

In [66]:
model.fit([x1, x2, x3], y, batch_size=64, nb_epoch=4)

Epoch 1/4
1289407/1289407 [==============================] - 89s - loss: 2.8506    
Epoch 2/4
1289407/1289407 [==============================] - 112s - loss: 2.8052   
Epoch 3/4
1289407/1289407 [==============================] - 118s - loss: 2.7578   
Epoch 4/4
1289407/1289407 [==============================] - 77s - loss: 2.7110    


In [67]:
get_next('She')

' '

In [68]:
get_next('Hol')

'e'

In [69]:
get_next('Wat')

' '